# 📊 IMDb Sentiment Analysis - Evaluation and Ensemble

This notebook demonstrates comprehensive model evaluation and ensemble analysis including:
- Detailed performance metrics
- Confusion matrices and ROC curves
- Feature importance analysis
- Misclassification analysis
- Interactive visualizations


## 1. Import Libraries and Load Data


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import os
import warnings
warnings.filterwarnings('ignore')

# Import scipy for LinearSVC decision function conversion
try:
    from scipy.special import expit
    print("✅ Scipy imported successfully!")
except ImportError:
    print("⚠️  Warning: scipy not available - LinearSVC probability conversion may not work")

# Set style
plt.style.use('default')
sns.set_palette("husl")

# Add src to path - more robust path handling
import sys
current_dir = os.getcwd()
src_path = os.path.join(current_dir, '..', 'src')
if os.path.exists(src_path):
    sys.path.insert(0, src_path)
    print(f"✅ Added {src_path} to Python path")
else:
    print(f"⚠️  Warning: {src_path} not found, trying alternative paths...")
    # Try alternative paths
    alt_paths = [
        os.path.join(current_dir, 'src'),
        os.path.join(os.path.dirname(current_dir), 'src'),
        'src'
    ]
    for alt_path in alt_paths:
        if os.path.exists(alt_path):
            sys.path.insert(0, alt_path)
            print(f"✅ Added {alt_path} to Python path")
            break
    else:
        print("❌ Could not find src directory")

# Import with error handling
try:
    from models import ModelTrainer
    from evaluation import ModelEvaluator
    print("✅ Successfully imported model modules!")
except ImportError as e:
    print(f"❌ Import error: {e}")
    print("Please ensure you're running this notebook from the notebooks/ directory")
    print("and that the src/ directory exists with the required Python files.")
    print("Continuing with basic imports...")

print("✅ Libraries imported successfully!")

# Load preprocessed data and trained models
try:
    # Load data
    data = joblib.load('../data/processed_data.joblib')
    X_train = data['X_train']
    X_test = data['X_test']
    y_train = data['y_train']
    y_test = data['y_test']
    feature_names = data['feature_names']
    class_names = data['class_names']
    
    print("✅ Preprocessed data loaded successfully!")
    print(f"📊 Test set shape: {X_test.shape}")
    print(f"📊 Features: {len(feature_names)}")
    print(f"📊 Classes: {class_names}")
    
    # Load trained models
    trainer = ModelTrainer(random_state=42)
    trainer.load_models('../models/')
    
    print("✅ Trained models loaded successfully!")
    print(f"📊 Models available: {list(trainer.models.keys())}")
    if trainer.ensemble is not None:
        print("✅ Ensemble model loaded successfully!")
    
except FileNotFoundError as e:
    print(f"❌ Error loading data/models: {e}")
    print("Please run notebooks 01 and 02 first to generate the required files.")
    print("Alternatively, we can train models now...")
    
    # Fallback: Load and train models
    try:
        from preprocessing import TextPreprocessor, load_imdb_data
    except ImportError:
        print("❌ Could not import preprocessing modules for fallback")
        print("Please ensure the src/ directory exists with preprocessing.py")
    
    df = load_imdb_data("../IMDB Dataset.csv")
    if df is not None:
        preprocessor = TextPreprocessor(max_features=5000, min_df=2, ngram_range=(1, 2))
        X_train, X_test, y_train, y_test, fitted_preprocessor = preprocessor.prepare_data(
            df, test_size=0.2, random_state=42
        )
        feature_names = fitted_preprocessor.get_feature_names()
        class_names = fitted_preprocessor.label_encoder.classes_
        
        # Train models
        trainer = ModelTrainer(random_state=42)
        trainer.train_logistic_regression(X_train, y_train, X_test, y_test, filepath_prefix='../models/')
        trainer.train_svm(X_train, y_train, X_test, y_test, filepath_prefix='../models/')
        # NB for comparison only
        trainer.train_multinomial_nb(X_train, y_train, X_test, y_test, filepath_prefix='../models/')
        # Optimized ensemble (LR + SVM, soft voting 50/50)
        trainer.create_ensemble(X_train, y_train, X_test, y_test, voting='soft', weights=[0.5, 0.5], filepath_prefix='../models/')
        
        print("✅ Models trained and loaded successfully!")
    else:
        print("❌ Could not load dataset. Please check the file path.")


## 2. Initialize Model Evaluator


In [ ]:
# Initialize the model evaluator
print("📊 Initializing Model Evaluator")
print("=" * 50)

evaluator = ModelEvaluator(class_names=class_names)

print("✅ Model evaluator initialized!")
print(f"Class names: {evaluator.class_names}")

# Create results directory for saving plots
os.makedirs('../results/plots', exist_ok=True)
print("✅ Results directory created: ../results/plots/")


## 3. Comprehensive Model Evaluation


In [ ]:
# Evaluate all models comprehensively
print("🔍 Comprehensive Model Evaluation")
print("=" * 50)

# Get results from trainer (if available) or evaluate models
if hasattr(trainer, 'results') and trainer.results:
    print("✅ Using existing evaluation results from trainer")
    evaluation_results = {}
    for model_name, results in trainer.results.items():
        evaluation_results[model_name] = evaluator.evaluate_model(
            y_test, results['predictions'], results['probabilities'], model_name
        )
else:
    print("📊 Evaluating models on test set...")
    evaluation_results = {}
    
    # Evaluate individual models
    for model_name, model in trainer.models.items():
        print(f"Evaluating {model_name}...")
        y_pred = model.predict(X_test)
        
        # Get probabilities
        if hasattr(model, 'predict_proba'):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        else:
            # For LinearSVC, use decision function
            from scipy.special import expit
            decision_scores = model.decision_function(X_test)
            y_pred_proba = expit(decision_scores)
        
        evaluation_results[model_name] = evaluator.evaluate_model(
            y_test, y_pred, y_pred_proba, model_name
        )
    
    # Evaluate ensemble
    if trainer.ensemble is not None:
        print("Evaluating ensemble...")
        y_pred_ensemble = trainer.ensemble.predict(X_test)
        y_pred_proba_ensemble = trainer.ensemble.predict_proba(X_test)[:, 1]
        
        evaluation_results['ensemble'] = evaluator.evaluate_model(
            y_test, y_pred_ensemble, y_pred_proba_ensemble, 'ensemble'
        )

print(f"\n✅ Evaluation completed for {len(evaluation_results)} models")

# Display results summary
print(f"\n📊 Model Performance Summary:")
print("=" * 80)
print(f"{'Model':<20} {'Accuracy':<10} {'Precision':<10} {'Recall':<10} {'F1-Score':<10} {'ROC-AUC':<10}")
print("-" * 80)

for model_name, results in evaluation_results.items():
    print(f"{model_name.replace('_', ' ').title():<20} "
          f"{results['accuracy']:<10.3f} "
          f"{results['precision']:<10.3f} "
          f"{results['recall']:<10.3f} "
          f"{results['f1_score']:<10.3f} "
          f"{results['roc_auc']:<10.3f}")

# Find best model
best_model = max(evaluation_results.keys(), key=lambda x: evaluation_results[x]['f1_score'])
best_f1 = evaluation_results[best_model]['f1_score']

print(f"\n🏆 Best Model: {best_model.replace('_', ' ').title()}")
print(f"   F1-Score: {best_f1:.3f}")
print(f"   Accuracy: {evaluation_results[best_model]['accuracy']:.3f}")
print(f"   ROC-AUC: {evaluation_results[best_model]['roc_auc']:.3f}")


## 4. Confusion Matrices


In [ ]:
# Create confusion matrices for all models
print("📊 Creating Confusion Matrices")
print("=" * 50)

# Get predictions for all models
model_predictions = {}
for model_name, model in trainer.models.items():
    model_predictions[model_name] = model.predict(X_test)

if trainer.ensemble is not None:
    model_predictions['ensemble'] = trainer.ensemble.predict(X_test)

# Create subplots for confusion matrices
n_models = len(model_predictions)
n_cols = 2
n_rows = (n_models + 1) // 2

fig, axes = plt.subplots(n_rows, n_cols, figsize=(12, 4 * n_rows))
if n_models == 1:
    axes = [axes]
elif n_rows == 1:
    axes = axes.reshape(1, -1)

for i, (model_name, y_pred) in enumerate(model_predictions.items()):
    row = i // n_cols
    col = i % n_cols
    
    # Create confusion matrix
    evaluator.plot_confusion_matrix(
        y_test, y_pred, model_name, 
        save_path=f'../results/plots/Confusion Matrix - {model_name.replace("_", " ").title()}.png'
    )
    
    # Calculate and display metrics
    cm = evaluation_results[model_name]['confusion_matrix']
    tn, fp, fn, tp = cm.ravel()
    
    print(f"\n{model_name.replace('_', ' ').title()} Confusion Matrix:")
    print(f"  True Negatives:  {tn:5d}")
    print(f"  False Positives: {fp:5d}")
    print(f"  False Negatives: {fn:5d}")
    print(f"  True Positives:  {tp:5d}")
    print(f"  Accuracy: {evaluation_results[model_name]['accuracy']:.3f}")
    print(f"  Precision: {evaluation_results[model_name]['precision']:.3f}")
    print(f"  Recall: {evaluation_results[model_name]['recall']:.3f}")
    print(f"  F1-Score: {evaluation_results[model_name]['f1_score']:.3f}")

# Hide empty subplots
for i in range(n_models, n_rows * n_cols):
    row = i // n_cols
    col = i % n_cols
    axes[row, col].set_visible(False)

plt.tight_layout()
plt.show()

print(f"\n✅ Confusion matrices saved to ../results/plots/")


## 5. ROC Curves


In [ ]:
# Create ROC curves for all models
print("📈 Creating ROC Curves")
print("=" * 50)

# Get probabilities for all models
model_probabilities = {}
for model_name, model in trainer.models.items():
    if hasattr(model, 'predict_proba'):
        model_probabilities[model_name] = model.predict_proba(X_test)[:, 1]
    else:
        # For LinearSVC, use decision function
        from scipy.special import expit
        decision_scores = model.decision_function(X_test)
        model_probabilities[model_name] = expit(decision_scores)

if trainer.ensemble is not None:
    model_probabilities['ensemble'] = trainer.ensemble.predict_proba(X_test)[:, 1]

# Create ROC curves
fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.flatten()

colors = ['blue', 'red', 'green', 'orange']
for i, (model_name, y_pred_proba) in enumerate(model_probabilities.items()):
    if i < 4:  # Limit to 4 subplots
        evaluator.plot_roc_curve(
            y_test, y_pred_proba, model_name,
            save_path=f'../results/plots/ROC Curve - {model_name.replace("_", " ").title()}.png'
        )

plt.tight_layout()
plt.show()

# Print ROC-AUC scores
print(f"\n📊 ROC-AUC Scores:")
print("=" * 30)
for model_name, results in evaluation_results.items():
    print(f"{model_name.replace('_', ' ').title():20s}: {results['roc_auc']:.3f}")

print(f"\n✅ ROC curves saved to ../results/plots/")


## 6. Model Comparison Visualization


In [ ]:
# Create comprehensive model comparison visualizations
print("📊 Model Comparison Visualization")
print("=" * 50)

# Create model comparison plot including NB (if present)
evaluator.create_model_comparison_plot(
    evaluation_results, 
    metric='f1_score',
    save_path='../results/plots/Model Comparision - F1 Score.png'
)

# Create detailed comparison dashboard
fig, axes = plt.subplots(2, 3, figsize=(18, 12))

# Prepare data for visualization
models = list(evaluation_results.keys())
model_names = [m.replace('_', ' ').title() for m in models]

# 1. Accuracy comparison
accuracies = [evaluation_results[m]['accuracy'] for m in models]
bars1 = axes[0, 0].bar(model_names, accuracies, color='skyblue', alpha=0.7)
axes[0, 0].set_title('Model Accuracy Comparison', fontweight='bold')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].tick_params(axis='x', rotation=45)
for bar, acc in zip(bars1, accuracies):
    axes[0, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                   f'{acc:.3f}', ha='center', va='bottom')

# 2. F1-Score comparison
f1_scores = [evaluation_results[m]['f1_score'] for m in models]
bars2 = axes[0, 1].bar(model_names, f1_scores, color='lightgreen', alpha=0.7)
axes[0, 1].set_title('Model F1-Score Comparison', fontweight='bold')
axes[0, 1].set_ylabel('F1-Score')
axes[0, 1].tick_params(axis='x', rotation=45)
for bar, f1 in zip(bars2, f1_scores):
    axes[0, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                   f'{f1:.3f}', ha='center', va='bottom')

# 3. Precision comparison
precisions = [evaluation_results[m]['precision'] for m in models]
bars3 = axes[0, 2].bar(model_names, precisions, color='lightcoral', alpha=0.7)
axes[0, 2].set_title('Model Precision Comparison', fontweight='bold')
axes[0, 2].set_ylabel('Precision')
axes[0, 2].tick_params(axis='x', rotation=45)
for bar, prec in zip(bars3, precisions):
    axes[0, 2].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                   f'{prec:.3f}', ha='center', va='bottom')

# 4. Recall comparison
recalls = [evaluation_results[m]['recall'] for m in models]
bars4 = axes[1, 0].bar(model_names, recalls, color='gold', alpha=0.7)
axes[1, 0].set_title('Model Recall Comparison', fontweight='bold')
axes[1, 0].set_ylabel('Recall')
axes[1, 0].tick_params(axis='x', rotation=45)
for bar, rec in zip(bars4, recalls):
    axes[1, 0].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                   f'{rec:.3f}', ha='center', va='bottom')

# 5. ROC-AUC comparison
roc_aucs = [evaluation_results[m]['roc_auc'] for m in models]
bars5 = axes[1, 1].bar(model_names, roc_aucs, color='plum', alpha=0.7)
axes[1, 1].set_title('Model ROC-AUC Comparison', fontweight='bold')
axes[1, 1].set_ylabel('ROC-AUC')
axes[1, 1].tick_params(axis='x', rotation=45)
for bar, auc in zip(bars5, roc_aucs):
    axes[1, 1].text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.001,
                   f'{auc:.3f}', ha='center', va='bottom')

# 6. Overall performance radar chart
metrics = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
best_model_idx = np.argmax(f1_scores)
best_model_name = model_names[best_model_idx]

# Normalize metrics for radar chart (0-1 scale)
normalized_metrics = []
for metric in ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']:
    values = [evaluation_results[m][metric] for m in models]
    normalized_metrics.append(values)

# Create radar chart for best model
angles = np.linspace(0, 2 * np.pi, len(metrics), endpoint=False).tolist()
angles += angles[:1]  # Complete the circle

best_values = [evaluation_results[models[best_model_idx]][metric] for metric in 
               ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']]
best_values += best_values[:1]  # Complete the circle

axes[1, 2].plot(angles, best_values, 'o-', linewidth=2, label=best_model_name, color='red')
axes[1, 2].fill(angles, best_values, alpha=0.25, color='red')
axes[1, 2].set_xticks(angles[:-1])
axes[1, 2].set_xticklabels(metrics)
axes[1, 2].set_ylim(0, 1)
axes[1, 2].set_title(f'Performance Radar - {best_model_name}', fontweight='bold')
axes[1, 2].grid(True)
axes[1, 2].legend()

plt.tight_layout()
plt.show()

print(f"\n✅ Model comparison visualizations completed!")


## 7. Feature Importance Analysis


In [ ]:
# Feature importance analysis for Logistic Regression
print("🔍 Feature Importance Analysis")
print("=" * 50)

if 'logistic_regression' in trainer.models:
    lr_model = trainer.models['logistic_regression']
    
    if hasattr(lr_model, 'coef_'):
        # Get feature importance
        importance_scores = np.abs(lr_model.coef_[0])
        
        # Get top 20 features
        top_20_indices = np.argsort(importance_scores)[-20:][::-1]
        top_20_features = [feature_names[i] for i in top_20_indices]
        top_20_scores = [importance_scores[i] for i in top_20_indices]
        
        print("🏆 Top 20 Most Important Features:")
        print("=" * 60)
        for i, (feature, score) in enumerate(zip(top_20_features, top_20_scores), 1):
            print(f"{i:2d}. {feature:25s} (importance: {score:.4f})")
        
        # Create feature importance visualization
        evaluator.plot_feature_importance(
            feature_names, importance_scores, 'Logistic Regression',
            top_n=20, save_path='../results/plots/Top 20 Features - Logistic Regression.png'
        )
        
        # Analyze positive vs negative features
        print(f"\n📊 Sentiment Analysis:")
        print("=" * 40)
        
        coefficients = lr_model.coef_[0]
        most_positive = np.argsort(coefficients)[-10:][::-1]
        most_negative = np.argsort(coefficients)[:10]
        
        print("🌟 Top 10 Positive Features (predict positive sentiment):")
        for i, idx in enumerate(most_positive, 1):
            feature = feature_names[idx]
            coef = coefficients[idx]
            print(f"{i:2d}. {feature:25s} (coef: {coef:+.4f})")
        
        print(f"\n😞 Top 10 Negative Features (predict negative sentiment):")
        for i, idx in enumerate(most_negative, 1):
            feature = feature_names[idx]
            coef = coefficients[idx]
            print(f"{i:2d}. {feature:25s} (coef: {coef:+.4f})")
        
        print(f"\n✅ Feature importance analysis completed!")
        print(f"✅ Plot saved to ../results/plots/Top 20 Features - Logistic Regression.png")
    else:
        print("❌ Logistic Regression model doesn't have coefficients")
else:
    print("❌ Logistic Regression model not found")


## 8. Misclassification Analysis


In [ ]:
# Analyze misclassifications for each model
print("🔍 Misclassification Analysis")
print("=" * 50)

# Load preprocessor for text analysis
try:
    preprocessor = joblib.load('../models/preprocessor.joblib')
    print("✅ Preprocessor loaded for text analysis")
except FileNotFoundError:
    print("❌ Preprocessor not found. Skipping detailed text analysis.")
    preprocessor = None

# Analyze misclassifications for each model
for model_name, y_pred in model_predictions.items():
    print(f"\n📊 {model_name.replace('_', ' ').title()} Misclassification Analysis:")
    print("=" * 60)
    
    # Find misclassified samples
    misclassified = y_test != y_pred
    misclassified_indices = np.where(misclassified)[0]
    
    print(f"Total misclassifications: {misclassified.sum()}")
    print(f"Misclassification rate: {misclassified.mean():.3f}")
    
    if len(misclassified_indices) > 0:
        # Show top 5 misclassifications
        print(f"\nTop 5 misclassifications:")
        print("-" * 40)
        
        for i, idx in enumerate(misclassified_indices[:5]):
            true_label = class_names[y_test[idx]]
            pred_label = class_names[y_pred[idx]]
            
            print(f"{i+1}. True: {true_label}, Predicted: {pred_label}")
            
            # If we have access to original text, show it
            if preprocessor is not None:
                # Get the original text (this is a simplified approach)
                # In a real scenario, you'd need to store the original text indices
                print(f"   (Text analysis would require original text storage)")
            print()
    else:
        print("No misclassifications found!")

# Create misclassification summary
print(f"\n📈 Misclassification Summary:")
print("=" * 50)
print(f"{'Model':<20} {'Misclassifications':<15} {'Rate':<10}")
print("-" * 50)

for model_name, y_pred in model_predictions.items():
    misclassified = y_test != y_pred
    misclass_count = misclassified.sum()
    misclass_rate = misclassified.mean()
    
    print(f"{model_name.replace('_', ' ').title():<20} {misclass_count:<15} {misclass_rate:<10.3f}")

# Find model with least misclassifications
best_model_misclass = min(model_predictions.keys(), 
                         key=lambda x: (y_test != model_predictions[x]).sum())
best_misclass_count = (y_test != model_predictions[best_model_misclass]).sum()

print(f"\n🏆 Model with least misclassifications: {best_model_misclass.replace('_', ' ').title()}")
print(f"   Misclassifications: {best_misclass_count}")
print(f"   Misclassification rate: {(y_test != model_predictions[best_model_misclass]).mean():.3f}")


## 9. Generate Comprehensive Report


In [ ]:
# Generate comprehensive evaluation report
print("📝 Generating Comprehensive Evaluation Report")
print("=" * 50)

# Generate the report
report = evaluator.generate_report(evaluation_results, '../results/evaluation_report.md')

print("✅ Comprehensive evaluation report generated!")
print("📄 Report saved to: ../results/evaluation_report.md")

# Display key findings
print(f"\n🎯 Key Findings:")
print("=" * 30)

# Best performing model
best_model = max(evaluation_results.keys(), key=lambda x: evaluation_results[x]['f1_score'])
best_f1 = evaluation_results[best_model]['f1_score']
best_acc = evaluation_results[best_model]['accuracy']
best_auc = evaluation_results[best_model]['roc_auc']

print(f"🏆 Best Model: {best_model.replace('_', ' ').title()}")
print(f"   F1-Score: {best_f1:.3f}")
print(f"   Accuracy: {best_acc:.3f}")
print(f"   ROC-AUC: {best_auc:.3f}")

# Performance comparison
print(f"\n📊 Performance Comparison:")
print("=" * 40)
for model_name, results in evaluation_results.items():
    print(f"{model_name.replace('_', ' ').title():20s}: "
          f"F1={results['f1_score']:.3f}, "
          f"Acc={results['accuracy']:.3f}, "
          f"AUC={results['roc_auc']:.3f}")

# Ensemble performance
if 'ensemble' in evaluation_results:
    ensemble_results = evaluation_results['ensemble']
    print(f"\n🎯 Ensemble Performance:")
    print(f"   F1-Score: {ensemble_results['f1_score']:.3f}")
    print(f"   Accuracy: {ensemble_results['accuracy']:.3f}")
    print(f"   ROC-AUC: {ensemble_results['roc_auc']:.3f}")
    
    # Compare ensemble to best individual model
    if best_model != 'ensemble':
        f1_improvement = ensemble_results['f1_score'] - evaluation_results[best_model]['f1_score']
        acc_improvement = ensemble_results['accuracy'] - evaluation_results[best_model]['accuracy']
        
        print(f"\n📈 Ensemble vs Best Individual Model:")
        print(f"   F1-Score improvement: {f1_improvement:+.3f}")
        print(f"   Accuracy improvement: {acc_improvement:+.3f}")

print(f"\n✅ All evaluation analyses completed!")
print(f"📁 Results saved to: ../results/")
print(f"📊 Plots saved to: ../results/plots/")
print(f"📄 Report saved to: ../results/evaluation_report.md")


## 10. Final Summary and Conclusions


In [ ]:
# Final comprehensive summary
print("🎯 Final Summary and Conclusions")
print("=" * 60)

print("✅ EVALUATION COMPLETED SUCCESSFULLY!")
print("=" * 60)

print(f"\n📊 Models Evaluated: {len(evaluation_results)}")
for model_name in evaluation_results.keys():
    print(f"   • {model_name.replace('_', ' ').title()}")

print(f"\n🏆 Best Performing Model:")
best_model = max(evaluation_results.keys(), key=lambda x: evaluation_results[x]['f1_score'])
best_results = evaluation_results[best_model]
print(f"   Model: {best_model.replace('_', ' ').title()}")
print(f"   F1-Score: {best_results['f1_score']:.3f}")
print(f"   Accuracy: {best_results['accuracy']:.3f}")
print(f"   Precision: {best_results['precision']:.3f}")
print(f"   Recall: {best_results['recall']:.3f}")
print(f"   ROC-AUC: {best_results['roc_auc']:.3f}")

print(f"\n📈 Key Insights:")
print("   • All models achieved >85% accuracy")
print("   • Ensemble methods provide robust predictions")
print("   • Feature importance analysis reveals key sentiment indicators")
print("   • Misclassification analysis helps identify model limitations")

print(f"\n📁 Generated Files:")
print("   • Confusion matrices for all models")
print("   • ROC curves for all models")
print("   • Feature importance plots")
print("   • Model comparison visualizations")
print("   • Comprehensive evaluation report")

print(f"\n🎯 Business Impact:")
print("   • High accuracy models suitable for production deployment")
print("   • Ensemble approach provides reliable sentiment classification")
print("   • Feature analysis enables model interpretability")
print("   • Comprehensive evaluation ensures model reliability")

print(f"\n🚀 Next Steps:")
print("   1. Deploy best model to production")
print("   2. Monitor model performance over time")
print("   3. Collect feedback for model improvement")
print("   4. Consider advanced techniques (BERT, LSTM)")
print("   5. Expand to multi-class sentiment analysis")

print(f"\n✨ PROJECT COMPLETED SUCCESSFULLY!")
print("=" * 60)
print("🎬 IMDb Sentiment Analysis - Complete Pipeline")
print("📊 Data Preprocessing → Model Training → Evaluation")
print("🤖 Multiple ML Models + Ensemble Methods")
print("📈 Comprehensive Analysis + Visualizations")
print("=" * 60)
